<a href="https://colab.research.google.com/github/zw2788/LocalMinimaConstruction/blob/main/LocalMinimaEx1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from typing import Tuple

import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from IPython.display import Image

In [3]:
data = pd.read_csv(
    "https://raw.githubusercontent.com/zw2788/LocalMinimaConstruction/main/Ex1.csv")

data.head()

,x_2dvec,y
0,"[2.8,0.4]",1
1,"[3.1,4.3]",1
2,"[0.1,-3.4]",1
3,"[-4.2,-3.3]",1
4,"[-0.5,0.2]",1


In [37]:
features = [
    "x_2dvec",

]
label = "y"

# train test split
X_raw,  Y = data[features].values, data[label].values

#convert string to array
X_raw = np.array([eval(s[0]) for s in X_raw])

# Standardize the input
# Leave blank to match the example in paper

# formatting
Y = Y.reshape((-1, 1))
print(X_raw)
print(Y)


[[ 2.8  0.4]
 [ 3.1  4.3]
 [ 0.1 -3.4]
 [-4.2 -3.3]
 [-0.5  0.2]
 [-2.7 -0.4]
 [-3.  -4.3]
 [-0.1  3.4]
 [ 4.2  3.2]
 [ 0.4 -0.1]]
[[1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [0]
 [0]]


Neural Structure:
$M((x_0, x_1)) = \sigma(v_0σ(w_{0,0}x_0 + w_{0,1}x_1 + b_0) + v_1σ(w_{1,0}x_0 + w_{1,1}x_1 + b_1) + c) $

In [9]:
Image(url='https://github.com/zw2788/LocalMinimaConstruction/blob/main/221Sigmoid.png?raw=true')
